In [1]:
import nengo
import nengo_dl
import tensorflow as tf
import numpy as np

import pickle
import gzip


In [2]:
train_data = []
test_data = []

# with gzip.open('train_data.pickle', 'rb') as f:
#     train_data = pickle.load(f)

with gzip.open('test_data.pickle', 'rb') as f:
    test_data = pickle.load(f)

In [3]:
test_data

[array([[-1.        , -1.        , -1.        , ..., -1.        ,
         -1.        , -1.        ],
        [-1.        , -1.        , -1.        , ..., -1.        ,
         -1.        , -1.        ],
        [-1.        , -1.        , -1.        , ..., -1.        ,
         -1.        , -1.        ],
        ...,
        [-0.99732078, -1.        , -1.        , ..., -1.        ,
         -1.        , -1.        ],
        [-0.96632561, -0.91662205, -1.        , ..., -1.        ,
         -1.        , -1.        ],
        [-0.98122171, -0.96365352, -1.        , ..., -1.        ,
         -1.        , -1.        ]]), array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])]

In [4]:
with nengo.Network() as net:
    # set some default parameters for the neurons that will make
    # the training progress more smoothly
    net.config[nengo.Ensemble].max_rates = nengo.dists.Choice([100])
#     net.config[nengo.Ensemble].intercepts = nengo.dists.Choice([0])
#     neuron_type = nengo.LIF(amplitude=0.01)
    neuron_type = nengo.SpikingRectifiedLinear()
    
    # we'll make all the nengo objects in the network
    # non-trainable. we could train them if we wanted, but they don't
    # add any representational power. note that this doesn't affect 
    # the internal components of tensornodes, which will always be 
    # trainable or non-trainable depending on the code written in 
    # the tensornode.
    nengo_dl.configure_settings(trainable=False)

    # the input node that will be used to feed in input images 
    inp = nengo.Node([0] * 16 * 16)

    # add the first convolutional layer
    x = nengo_dl.tensor_layer(
        inp, tf.layers.conv2d, shape_in=(16, 16, 1), filters=16,
        kernel_size=3)

    # apply the neural nonlinearity
    x = nengo_dl.tensor_layer(x, neuron_type)
# filters랑 shape_in의 마지막 숫자랑 맞춰줘야해 
# kernel size만큼 돌면 남는게 output임
    # add another convolutional layer
    x = nengo_dl.tensor_layer(
        x, tf.layers.conv2d, shape_in=(14, 14, 16), filters=14, kernel_size=3)
    x = nengo_dl.tensor_layer(x, neuron_type)

    # add a pooling layer
    x = nengo_dl.tensor_layer(
        x, tf.layers.average_pooling2d, shape_in=(12, 12, 14),
        pool_size=2, strides=2)

    # another convolutional layer
    x = nengo_dl.tensor_layer(
        x, tf.layers.conv2d, shape_in=(6, 6, 14),
        filters=28, kernel_size=3)
    x = nengo_dl.tensor_layer(x, neuron_type)

    # another pooling layer
    x = nengo_dl.tensor_layer(
        x, tf.layers.average_pooling2d, shape_in=(4, 4, 28),
        pool_size=2, strides=2)


    # linear readout
    x = nengo_dl.tensor_layer(x, tf.layers.dense, units=31)

    # we'll create two different output probes, one with a filter
    # (for when we're simulating the network over time and
    # accumulating spikes), and one without (for when we're
    # training the network using a rate-based approximation)
    out_p = nengo.Probe(x)
    out_p_filt = nengo.Probe(x, synapse=0.1)

In [5]:
minibatch_size = 16
sim = nengo_dl.Simulator(net, minibatch_size=minibatch_size)

Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               #########################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################################

In [6]:
def objective(outputs, targets): 
    return tf.nn.softmax_cross_entropy_with_logits_v2(
        logits=outputs, labels=targets)

In [7]:
opt = tf.train.RMSPropOptimizer(learning_rate=0.0001)

In [8]:
train_data_array = {inp: test_data[0][:, None, :],
              out_p: test_data[1][:, None, :]}

In [9]:
n_steps = 30
test_data_array = {
    inp: np.tile(test_data[0][:minibatch_size*2, None, :],
                 (1, n_steps, 1)),
    out_p_filt: np.tile(test_data[1][:minibatch_size*2, None, :],
                        (1, n_steps, 1))}

In [10]:
def classification_error(outputs, targets):
    return 100 * tf.reduce_mean(
        tf.cast(tf.not_equal(tf.argmax(outputs[:, -1], axis=-1), 
                             tf.argmax(targets[:, -1], axis=-1)),
                tf.float32))


print("error before training: %.2f%%" % sim.loss(
    test_data_array, {out_p_filt: classification_error}))

Calculation finished in 0:00:01                                                
error before training: 90.62%


In [11]:
do_training = True
if do_training:
    # run training
    sim.train(train_data_array, opt, objective={out_p: objective}, n_epochs=10)
    
    # save the parameters to file
    sim.save_params("model_output/nsmc_params")

|                   Training (0%)                  | ETA:  --:--:-- (loss: ---)

/home/banana/.local/lib/python3.5/site-packages/nengo_dl/utils.py:489: UserWarning: Number of data elements (632153) is not an even multiple of minibatch size (16); inputs will be truncated
  (n_inputs, minibatch_size)))


Training finished in 0:03:37 (loss: 46.2389)                                   


KeyboardInterrupt: 

In [ ]:
print("error after training: %.2f%%" % sim.loss(
    test_data_array, {out_p_filt: classification_error}))